# Fig. S19-23
Significant sustained electrodes, sequence complexity, and ERPs of interest for participants who were excluded from some analyses for various reasons (EC267, EC276, EC282, and EC289). As 4 of these participants had stimulation, we also plot the same figure for EC260 (the last stimulation participant) even though they were included in the main text figures. See Table S1 for more details.

In [ ]:
import json
import os
import copy
import pickle
from datetime import datetime

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from matplotlib.lines import Line2D

from sylseq_paper.plotting import default_plot_settings, ucsf_sequential_color_palette as colors
from sylseq_paper.stimulation_info import stim_pairs

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

default_plot_settings(font='Helvetica', fontsize=14)

In [ ]:
# Change this to plot one of the 5 subjects
subject = 'EC289'

erp_elecs = {
    'EC260': [76, 204, 82],
    'EC267': [230, 66],
    'EC276': [53, 157],
    'EC282': [143],
    'EC289': [36, 4]
}
erp_plot_periods = {
    'EC260': ['tp_and_delay', 'speech'],
    'EC267': ['tp', 'delay', 'speech'],
    'EC276': ['tp_and_delay', 'speech'],
    'EC282': ['tp_and_delay', 'speech'],
    'EC289': ['tp_and_delay', 'speech']
}

sig_thresh = 0.05
exclude_areas = ['superiorparietal', 'inferiortemporal', 'paracentral', 'medial']

# Set up paths
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
img_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'imaging'))
subj_img_path = os.path.join(img_dir, '{}_{}_brain_2D.png')

# Load bad channels
with open(os.path.join(data_dir, 'bad_channels.pkl'), 'rb') as f:
    bad_channels = pickle.load(f)
    
bad_ch = np.unique(np.concatenate(list(bad_channels[subject].values())))
    
# Load elec info
all_edf = pd.read_hdf(os.path.join(data_dir, 'all_anatomical_info.h5'))
all_edf = all_edf.loc[~all_edf.location.isin(exclude_areas)]

In [ ]:
seq_vs_syl_df = pd.read_hdf(os.path.join(data_dir, 'fig3_seq_vs_syl_df.h5'))
seq_vs_syl_df['subject_electrode'] = [f'{s}_{e}' for s, e in zip(seq_vs_syl_df.subject.values, seq_vs_syl_df.electrode.values)]
seq_vs_syl_df = seq_vs_syl_df.loc[seq_vs_syl_df.subject == subject]

seq_syl_colors = {
    'sequence': colors[0],
    'syllable': colors[3],
    'both': colors[2]
}

if subject == 'EC289':
    complexity_period = 'delay'
else:
    complexity_period = 'pre_exec'
    
period_labels = {
    'delay': 'Delay',
    'pre_exec': 'Pre-speech'
}
    
seq_elecs = seq_vs_syl_df.loc[(seq_vs_syl_df.alignment == complexity_period) &
                              (seq_vs_syl_df.fdr_pval < sig_thresh) &
                              (seq_vs_syl_df.feature_type == 'sequence')].electrode.values
syl_elecs = seq_vs_syl_df.loc[(seq_vs_syl_df.alignment == complexity_period) &
                              (seq_vs_syl_df.fdr_pval < sig_thresh) &
                              (seq_vs_syl_df.feature_type == 'syllable')].electrode.values
all_effect_elecs = np.union1d(seq_elecs, syl_elecs)
both_elecs = np.intersect1d(seq_elecs, syl_elecs)
seq_elecs = np.setdiff1d(seq_elecs, both_elecs)
syl_elecs = np.setdiff1d(syl_elecs, both_elecs)

In [ ]:
# Load dataframe electrode information
with open(os.path.join(data_dir, 'fig1_ndf_sustained_se.pkl'), 'rb') as f:
    results = pickle.load(f)
    neural_df = results['neural_df']
    main_sustained_se = results['sustained_se']
    
if subject in neural_df.subject.unique():
    any_period_sig_elecs = neural_df.loc[(neural_df.subject == subject) &
                  (neural_df.significant_above_baseline_anytime)].electrode.unique()
    sustained_elecs = np.array([int(se.split('_')[1]) for se in main_sustained_se if se.startswith(subject)])
    complexity_period_above_baseline = neural_df.loc[(neural_df.subject == subject) & 
                                                     (neural_df.alignment == complexity_period) & 
                                                     (neural_df.ranksum_above_baseline_significant)].electrode.values

else:
    with open(os.path.join(data_dir, f'{subject}_supp_analyses.pkl'), 'rb') as f:
        results = pickle.load(f)
        any_period_sig_elecs = results['any_period_sig_elecs']
        sustained_elecs = results['sustained_elecs']
        complexity_period_above_baseline = results['per_period_sig_elecs'][complexity_period]
        
# Load ERPs
with open(os.path.join(data_dir, f'{subject}_suppfig_erps.pkl'), 'rb') as f:
    seq_complexity_trials = pickle.load(f)

In [ ]:
def plot_recon(view='lateral', highlight_elecs=False, plot_seq_complexity=False, ax=None, return_fig=False,
               fs=10, s=20, highlight_stim=None, plot_legend=True, plot_seq_complexity_only=False):
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
        
    edf = all_edf.loc[all_edf.subject == subject]
    edf = edf.loc[(edf.view == view) & (~edf.location.isin(exclude_areas))]
        
    img = plt.imread(os.path.join(img_dir, f'{subject}_{view}_brain_2D.png'))
    ax.imshow(img, alpha=0.5, zorder=1)
    
    if plot_seq_complexity:
        pre_exec_sig_edf = edf.loc[(edf.electrode.isin(complexity_period_above_baseline)) &
                                   (~edf.electrode.isin(all_effect_elecs)) & 
                                   (~edf.electrode.isin(bad_ch))]
        seq_edf = edf.loc[edf.electrode.isin(seq_elecs)]
        syl_edf = edf.loc[edf.electrode.isin(syl_elecs)]
        both_edf = edf.loc[edf.electrode.isin(both_elecs)]
        
        ax.scatter(pre_exec_sig_edf.x, pre_exec_sig_edf.y, s=s//4, color='k', alpha=0.3, zorder=3,
                  label=f'{period_labels[complexity_period]} sig.')
        ax.scatter(seq_edf.x, seq_edf.y, s=s, facecolor=seq_syl_colors['sequence'], alpha=1, zorder=3,
                  label='Seq. complexity')
        ax.scatter(syl_edf.x, syl_edf.y, s=s, facecolor=seq_syl_colors['syllable'], alpha=1, zorder=3,
                  label='Artic. complexity')
        ax.scatter(both_edf.x, both_edf.y, s=s, facecolor=seq_syl_colors['both'], alpha=1, zorder=3,
                  label='Seq. & artic. complexity')
        legend_col = 2
        
    elif plot_seq_complexity_only:
        pre_exec_sig_edf = edf.loc[(edf.electrode.isin(complexity_period_above_baseline)) &
                                   (~edf.electrode.isin(all_effect_elecs)) & 
                                   (~edf.electrode.isin(bad_ch))]
        seq_edf = edf.loc[edf.electrode.isin(seq_elecs) | edf.electrode.isin(both_elecs)]
        
        ax.scatter(pre_exec_sig_edf.x, pre_exec_sig_edf.y, s=s//4, color='k', alpha=0.3, zorder=3,
                  label=f'{period_labels[complexity_period]} sig.')
        ax.scatter(seq_edf.x, seq_edf.y, s=s, facecolor=seq_syl_colors['sequence'], alpha=1, zorder=3,
                  label='Sequence complexity')
        legend_col = 2
        
    else:
        nonsig_edf = edf.loc[~edf.electrode.isin(any_period_sig_elecs) & (~edf.electrode.isin(bad_ch))]
        sig_edf = edf.loc[(edf.electrode.isin(any_period_sig_elecs)) & (~edf.electrode.isin(sustained_elecs))]
        sig_sustained_edf = edf.loc[edf.electrode.isin(sustained_elecs)]
    
        ax.scatter(nonsig_edf.x, nonsig_edf.y, s=s//4, color='k', alpha=0.3, zorder=3,
                   label='Not sig.')
        ax.scatter(sig_edf.x, sig_edf.y, s=s, facecolor=colors[1], edgecolor='None', alpha=0.6, 
                   label='Sig. during any period', zorder=3)
        ax.scatter(sig_sustained_edf.x, sig_sustained_edf.y, s=s, facecolor=colors[1], edgecolor='k', 
                   label='Sustained', alpha=0.6, zorder=3)
        legend_col = 3
        
    handles, labels = ax.get_legend_handles_labels()
        
    if highlight_elecs or type(highlight_elecs) is int:
        
        if type(highlight_elecs) is int:
            cur_edf = edf.loc[edf.electrode == erp_elecs[subject][highlight_elecs]]
        else:
            cur_edf = edf.loc[edf.electrode.isin(erp_elecs[subject])]
            
        ax.scatter(cur_edf.x, cur_edf.y, s=s, facecolor='None', edgecolor='k', zorder=3)
        
    if highlight_stim:
        
        for key, val in stim_pairs[subject].items():
            
            # correct for 1-indexing
            a, b = np.array(val['research_elecs']) - 1
            A = edf.loc[edf.electrode == a].iloc[0]
            B = edf.loc[edf.electrode == b].iloc[0]
            
            linestyle = '-'
            
            if val['seq_deficit'] and not val['inconsistent_seq_deficit'] and not val['motor_deficit']:
                linecolor = colors[1]
            elif not val['seq_deficit'] and not val['inconsistent_seq_deficit']:
                # No sequence effect
                # Elecs with no sequence effect but motor at current or higher amplitudes
                if val['motor_deficit'] or ('higher_amp_motor_deficit' in val.keys() and val['higher_amp_motor_deficit'] == True):
                    linecolor = colors[4]
                
                elif not val['motor_deficit'] or ('higher_amp_motor_deficit' in val.keys() and val['higher_amp_motor_deficit'] in [False, 'not_tested']):
                    if 'other_deficit' in val.keys():
                        # Elecs with no motor deficits but other deficits (perceptual or sensory)
                        linecolor = colors[4]
                    else:
                        # Elecs with no motor deficits and no other deficits
                        linecolor = 'grey'
                        linestyle = '-'
            
            if subject == 'EC276' and key == 'PG11-PG13':
                # Just shifting the line slightly because it is overlapping with another
                ax.plot([A.x + 5, B.x + 5], [A.y, B.y], color=linecolor, alpha=1, linestyle=linestyle, linewidth=2, zorder=2)
            elif subject == 'EC260' and key == 'FG11-FG13':
                # Just shifting the line slightly because it is overlapping with another
                ax.plot([A.x + 0.5, B.x], [A.y + 5, B.y + 5], color=linecolor, alpha=1, linestyle=linestyle, linewidth=2, zorder=2)
            else:
                ax.plot([A.x, B.x], [A.y, B.y], color=linecolor, alpha=1, linestyle=linestyle, linewidth=2, zorder=2)
            
        handles.append(Line2D([0], [0], color=colors[1], lw=2))
        handles.append(Line2D([0], [0], color=colors[4], lw=2))
        handles.append(Line2D([0], [0], color='grey', lw=2))
        labels.extend(['Sequencing errors', 'Other sensorimotor effects', 'No deficits'])
    
    if plot_legend:
        ax.legend(handles, labels, frameon=False, fontsize=fs, ncol=legend_col, 
                  columnspacing=0.5, bbox_to_anchor=(0.5, -0.05), loc='upper center')
    
    ax.axis('off')
    
    if return_fig:
        return fig, ax
    else:
        return ax

In [ ]:
plot_recon();

In [ ]:
plot_recon(plot_seq_complexity=True, highlight_elecs=True, highlight_stim=True);

In [ ]:
if subject == 'EC282':
    seq_complexity_conditions = {
        'CV': {'sequence_type': 'none', 'syllable_type': 'simple'},
        'Repeated CV': {'sequence_type': 'simple', 'syllable_type': 'simple'},
        'CV sequence': {'sequence_type': 'complex', 'syllable_type': 'simple'}
    }
elif subject == 'EC289':
    seq_complexity_conditions = {
        'Repeated CV': {'sequence_type': 'simple', 'syllable_type': 'simple'},
        'Repeated CCV': {'sequence_type': 'simple', 'syllable_type': 'complex'},
        'CV sequence': {'sequence_type': 'complex', 'syllable_type': 'simple'},
        'CCV sequence': {'sequence_type': 'complex', 'syllable_type': 'complex'}
    }
elif subject == 'EC260':
    seq_complexity_conditions = {
        'CV': {'sequence_type': 'none', 'syllable_type': 'simple'},
        'CCV': {'sequence_type': 'none', 'syllable_type': 'complex'},
        'Repeated CV': {'sequence_type': 'simple', 'syllable_type': 'simple'},
        'Repeated CCV': {'sequence_type': 'simple', 'syllable_type': 'complex'},
        'CV sequence': {'sequence_type': 'complex', 'syllable_type': 'simple'},
        'CCV sequence': {'sequence_type': 'complex', 'syllable_type': 'complex'}
    }

else:
    seq_complexity_conditions = {
        'CV': {'sequence_type': 'none', 'syllable_type': 'simple'},
        'Repeated CV': {'sequence_type': 'simple', 'syllable_type': 'simple'},
        'CV sequence': {'sequence_type': 'complex', 'syllable_type': 'simple'},
        'CCV sequence': {'sequence_type': 'complex', 'syllable_type': 'complex'}
    }

seq_complexity_labels = {
    'CV': 'Simple syl.\n$\it{baa}$',
    'CCV': 'Complex syl.\n$\it{blaa}$',
    'Repeated CV': 'Simple syl., simple seq.\n$\it{baa}$-$\it{baa}$-$\it{baa}$',
    'Repeated CCV': 'Complex syl., simple seq.\n$\it{blaa}$-$\it{blaa}$-$\it{blaa}$',
    'CV sequence': 'Simple syl., complex seq.\n$\it{baa}$-$\it{daa}$-$\it{gaa}$',
    'CCV sequence': 'Complex syl., complex seq.\n$\it{blaa}$-$\it{draa}$-$\it{gloo}$'
}
complexity_colors = {
    'CV': colors[2],
    'CCV': colors[7],
    'Repeated CV': colors[4],
    'Repeated CCV': 'sienna',
    'CV sequence': colors[1],
    'CCV sequence': colors[5]
}

alignments = {
    'tp_and_delay': {
        'window': [-0.5, 2.5 + 1],
        'phase': 'target_presentation',
        'phase_letter': 'p'
    },
    'tp': {
        'window': [-0.5, 0.75],
        'phase': 'target_presentation',
        'phase_letter': 'p'
    },
    'delay': {
        'window': [-0.5, 0.75],
        'phase': 'fixation_cue',
        'phase_letter': 'f'
    },
    'speech': {
        'window': [-0.5, 1.0],
        'phase': 'first_syllable',
        'phase_letter': 'e'
    }
}

In [ ]:
def complexity_erps(axs=None, elec=None, plot_periods=None, return_fig=False, plot_legend=False, 
                    fs=10, annotate_lines=True, plot_conditions=None):
    
    width_ratios = []
    for period in plot_periods:
        width_ratios.append(np.sum(np.abs(np.array(alignments[period]['window']))))
    
    if axs is None:
        fig, axs = plt.subplots(1, len(plot_periods), figsize=(12, 4),
                                gridspec_kw=dict(width_ratios=width_ratios))
        
    erp_plot_kwargs = {'ylim': [-1, 3]}
    
    if subject == 'EC282':
        erp_plot_kwargs = {'ylim': [-1, 4]}
    elif subject == 'EC289':
        erp_plot_kwargs = {'ylim': [-1, 4]}
    
    erp_axline_kwargs = {
        'linestyle': '--',
        'linewidth': 1,
        'color': 'k'
    }
        
    axs[0].axes.set_ylabel('HGA (z-score)', fontsize=fs)
        
    for cur_ax, (alignment_label, ax) in enumerate(zip(plot_periods, axs.ravel())):

        if alignment_label == 'tp_and_delay':
            ax.axvline(x=0, alpha=0.7, zorder=1, **erp_axline_kwargs)
            ax.axvline(x=2.5, alpha=0.7, zorder=1, **erp_axline_kwargs)
            
            if annotate_lines:
                ax.annotate('Target\npres.', (0.0, erp_plot_kwargs['ylim'][1]), ha='center', va='bottom', fontsize=fs)
                ax.annotate('Fixation\ncross', (2.5, erp_plot_kwargs['ylim'][1]), ha='center', va='bottom', fontsize=fs)

        elif alignment_label == 'tp':
            ax.axvline(x=0, alpha=0.7, zorder=1, **erp_axline_kwargs)
            
            if annotate_lines:
                ax.annotate('Target\npres.', (0.0, erp_plot_kwargs['ylim'][1]), ha='center', va='bottom', fontsize=fs)
            
        elif alignment_label == 'delay':
            ax.axvline(x=0, alpha=0.7, zorder=1, **erp_axline_kwargs)
            
            if annotate_lines:
                ax.annotate('Fixation\ncross', (0.0, erp_plot_kwargs['ylim'][1]), ha='center', va='bottom', fontsize=fs)
        
        elif alignment_label == 'speech':
            ax.axvline(x=0, alpha=0.7, zorder=1, **erp_axline_kwargs)
            
            if annotate_lines:
                ax.annotate('Speech\nonset', (0.0, erp_plot_kwargs['ylim'][1]), ha='center', va='bottom', fontsize=fs)

        if cur_ax == 0:
            despine_kw = dict(offset=dict(bottom=5, left=5))
        else:
            despine_kw = dict(offset=dict(bottom=5, left=5), left=True)
        sns.despine(ax=ax, **despine_kw)

        if cur_ax != 0:
            ax.get_yaxis().set_visible(False)

        ax.axhline(y=0, alpha=0.7, zorder=1, **erp_axline_kwargs)

        # Set major xticks at 1, minor at 0.5
        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.xaxis.set_minor_locator(MultipleLocator(0.25))
        ax.yaxis.set_major_locator(MultipleLocator(2))
        ax.yaxis.set_minor_locator(MultipleLocator(1))

        for cur_cond, (cond_label, ecog_trials) in enumerate(seq_complexity_trials[subject][alignment_label].items()):
            
            if plot_conditions is not None and cond_label not in plot_conditions:
                continue

            y = np.nanmean(ecog_trials[:, :, elec], axis=0)               
            err = stats.sem(ecog_trials[:, :, elec], axis=0, nan_policy='omit')
            x = np.linspace(alignments[alignment_label]['window'][0], alignments[alignment_label]['window'][1], y.shape[0])
            ax.plot(x, y, color=complexity_colors[cond_label], clip_on=False, label=seq_complexity_labels[cond_label], zorder=2)
            ax.fill_between(x, y - err, y + err, color=complexity_colors[cond_label], alpha=0.2, clip_on=False, zorder=2)

        ax.axes.set(xlim=alignments[alignment_label]['window'], **erp_plot_kwargs)
        ax.set_xlabel('Time (s)', fontsize=fs)
            
    an_kw = dict(fontsize=18, annotation_clip=False)
    
    if plot_legend:
        axs[-1].legend(loc='upper left', bbox_to_anchor=(1, 1), frameon=False, fontsize=fs, ncol=1)
    
    if return_fig:
        return fig, axs
    else:
        return axs

# Overall figure

In [ ]:
##### Figure setup #####
default_plot_settings(font='Helvetica', linewidth=1.5)

all_axs = {}

if subject == 'EC260':
    fig = plt.figure(figsize=(10, 10))
    fs = 8
else:
    fig = plt.figure(figsize=(10, 10))
    fs = 10
    
gs = mpl.gridspec.GridSpec(100, 100, figure=fig)

width_ratios = []
for period in erp_plot_periods[subject]:
    width_ratios.append(np.sum(np.abs(np.array(alignments[period]['window']))))
        
if subject == 'EC282':
    brain_lim = 40
    erp_gs = mpl.gridspec.GridSpecFromSubplotSpec(1, len(erp_plot_periods[subject]), 
                                                  subplot_spec=gs[55:77, 2:75], 
                                                  width_ratios=width_ratios,
                                                  hspace=0.5,
                                                  wspace=0.1)
    
    
elif subject == 'EC260':
    brain_lim = 33
    erp_gs = mpl.gridspec.GridSpecFromSubplotSpec(3, len(erp_plot_periods[subject]), 
                                                  subplot_spec=gs[brain_lim+14:, 2:75], 
                                                  width_ratios=width_ratios,
                                                  hspace=0.75,
                                                  wspace=0.1)
    
    
else:
    brain_lim = 40
    erp_gs = mpl.gridspec.GridSpecFromSubplotSpec(2, len(erp_plot_periods[subject]), 
                                                  subplot_spec=gs[55:, 2:75], 
                                                  width_ratios=width_ratios,
                                                  hspace=0.5,
                                                  wspace=0.1)
    

########################

# Plot recons
if subject in ['EC276', 'EC289']:
    all_axs['sig_elecs_lateral'] = fig.add_subplot(gs[4:40, 11:48])
    all_axs['sig_elecs_lateral'] = plot_recon(view='lateral', ax=all_axs['sig_elecs_lateral'], fs=fs);
    
    all_axs['sig_elecs_medial'] = fig.add_subplot(gs[:15, :20])
    all_axs['sig_elecs_medial'] = plot_recon(view='medial', ax=all_axs['sig_elecs_medial'], plot_legend=False, fs=fs);
    
    all_axs['seq_elecs_lateral'] = fig.add_subplot(gs[4:40, 63:])
    all_axs['seq_elecs_lateral'] = plot_recon(view='lateral', plot_seq_complexity=True, ax=all_axs['seq_elecs_lateral'],
                                      highlight_elecs=True, highlight_stim=True, fs=fs);
    
    all_axs['seq_elecs_medial'] = fig.add_subplot(gs[:15, 52:72])
    all_axs['seq_elecs_medial'] = plot_recon(view='medial', plot_seq_complexity=True, ax=all_axs['seq_elecs_medial'],
                                             highlight_elecs=True, highlight_stim=False, plot_legend=False, fs=fs);

else:
    all_axs['sig_elecs_lateral'] = fig.add_subplot(gs[:brain_lim, :45])
    all_axs['sig_elecs_lateral'] = plot_recon(view='lateral', ax=all_axs['sig_elecs_lateral'], fs=fs);
    
    all_axs['seq_elecs_lateral'] = fig.add_subplot(gs[:brain_lim, 55:])
    all_axs['seq_elecs_lateral'] = plot_recon(view='lateral', plot_seq_complexity=True, ax=all_axs['seq_elecs_lateral'],
                                      highlight_elecs=True, highlight_stim=True, fs=fs);
    
# ERP examples
for cur_erp in range(len(erp_elecs[subject])):
    all_axs[f'erp{cur_erp}'] = np.array([fig.add_subplot(erp_gs[cur_erp, i]) for i in range(len(erp_plot_periods[subject]))])
    all_axs[f'erp{cur_erp}'] = complexity_erps(plot_periods=erp_plot_periods[subject], 
                                               elec=cur_erp, 
                                               plot_legend=cur_erp == 0,
                                               annotate_lines=cur_erp == 0,
                                               axs=all_axs[f'erp{cur_erp}'])
    

############
if subject in ['EC276', 'EC289']:
    all_axs['sig_elecs_medial'].annotate('a', (-0.05, 0.85), xycoords='axes fraction', ha='right', fontsize=18, weight='bold');
    all_axs['seq_elecs_medial'].annotate('b', (-0.05, 0.85), xycoords='axes fraction', ha='right', fontsize=18, weight='bold');
    
else:
    all_axs['sig_elecs_lateral'].annotate('a', (-0.05, 1), xycoords='axes fraction', ha='right', fontsize=18, weight='bold');
    all_axs['seq_elecs_lateral'].annotate('b', (-0.05, 1), xycoords='axes fraction', ha='right', fontsize=18, weight='bold');
    
all_axs['erp0'][0].annotate('c', (-0.1, 1.2), xycoords='axes fraction', ha='right', fontsize=18, weight='bold');